<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/Cadec_twadr_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,57,hypertrophic scarring
3,85,tremor
4,94,anxiety


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-5.test.csv",header=None)
test.head()

,0,1
0,13,up all night
1,13,insomnia
2,93,shooting
3,133,night terrors
4,231,lucid dreamer


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'of',
 'in',
 'a',
 'it',
 'me',
 'mood',
 'increased',
 'have',
 'for',
 '-',
 'was',
 'drug',
 'lipitor',
 ')',
 '(',
 'this',
 'on',
 'disorder',
 'not',
 'abnormal',
 'withdrawal',
 'symptoms',
 'anxiety',
 'had',
 'that',
 'is',
 'with',
 'muscle',
 'like',
 'weight',
 'but',
 'feeling',
 "n't",
 'decreased',
 'taking',
 'disorders',
 'sleeplessness',
 'behavior',
 'after',
 'up',
 'syndrome',
 'sleep',
 'got',
 'blood',
 'so',
 'all',
 'am',
 'insomnia',
 'diseases',
 'feel',
 'energy',
 'side',
 'from',
 'depression',
 'somnolence',
 'disturbance',
 'finding',
 'back',
 'been',
 'loss',
 '!',
 'effects',
 'day',
 'mental',
 'be',
 'cholesterol',
 'heart',
 'skin',
 'as',
 'gain',
 'acute',
 'appetite',
 'body',
 'take',
 'nausea',
 '/',
 'at',
 'has',
 'about',
 'no',
 'mg',
 'off',
 'out',
 'or',
 'dizziness',
 'years',
 'do',
 'fat

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.302203,3.456800,0.427902,08:03


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.270485,3.024164,0.486021,08:04


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.052448,2.986059,0.490123,08:02


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.030931,2.954118,0.493485,08:05


In [25]:
learn.save_encoder('TwADR-cadec_fold5_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.175142,6.471136,0.092308,00:24


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.228246,5.176633,0.138462,00:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.695959,4.852315,0.176923,01:02


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.421058,4.604723,0.200000,01:00
1,5.202295,4.449736,0.238462,00:57


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.917203,4.400030,0.215385,00:59
1,4.734728,4.255698,0.261538,00:57


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.559285,4.113856,0.246154,01:02
1,4.389667,3.941091,0.284615,01:00


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.161315,3.881811,0.315385,00:59
1,4.003309,3.855525,0.300000,00:57


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.844243,3.957404,0.269231,01:02
1,3.695170,3.833563,0.269231,00:59


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.542506,3.815390,0.315385,00:59
1,3.425995,3.720513,0.323077,01:02
2,3.245570,3.693563,0.315385,01:03
3,3.101178,3.697041,0.307692,00:58


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.922695,3.681386,0.315385,00:57
1,2.877566,3.768592,0.330769,01:02
2,2.687404,3.627576,0.323077,01:02
3,2.548223,3.603961,0.307692,01:03


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.412313,3.607807,0.346154,00:58
1,2.368628,3.863670,0.353846,01:03
2,2.202059,3.667300,0.338462,01:04
3,2.045369,3.722584,0.323077,01:00


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.944073,3.750135,0.361538,00:58
1,1.869934,3.788470,0.315385,01:05
2,1.735583,3.740288,0.369231,01:03
3,1.624355,3.727159,0.353846,00:59


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.502259,3.750790,0.323077,01:04
1,1.524301,3.994105,0.353846,00:57
2,1.400871,3.940380,0.353846,01:01
3,1.296710,3.962414,0.346154,00:59


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.185630,4.028072,0.369231,00:58
1,1.204255,4.187693,0.346154,00:57
2,1.118299,4.182107,0.346154,00:59
3,1.017137,4.075468,0.369231,00:59


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.942349,4.082860,0.353846,01:00
1,0.950671,4.034656,0.353846,01:00
2,0.895813,4.234608,0.353846,01:00
3,0.808590,4.200587,0.353846,00:59


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.788092,4.320351,0.353846,01:00
1,0.752061,4.280685,0.346154,01:05


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.747716,4.277593,0.346154,01:02
1,0.688521,4.336709,0.353846,01:05


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.654662,4.318522,0.384615,01:02


In [44]:
# save the best model

learn.save_encoder('TwADR-cadec_fold5')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.651300,4.439783,0.376923,00:58


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.661499,4.368897,0.361538,01:01


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.617390,4.478823,0.338462,01:01


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1099, 393, 1787, 133, 231, 233, 2147, 249, 300, 1582, 1718, 824, 418, 532, 550, 1373, 582, 749, 644, 744, 744, 749, 749, 1597, 1597, 1597, 792, 792, 792, 813, 1940, 825, 822, 1278, 1278, 858, 441, 974, 974, 974, 978, 978, 978, 978, 978, 978, 1007, 1019, 1054, 2191, 2147, 2191, 1227, 1136, 1142, 1175, 1192, 561, 1227, 1227, 1295, 1295, 1295, 1308, 1340, 441, 947, 452, 1325, 1595, 1595, 809, 1718, 1341, 1508, 1468, 224, 418, 1701, 1711, 749, 1925, 1163, 1356, 1597, 1597, 1610, 1610, 1610, 1610, 1610, 1610, 1145, 1048, 1701, 749, 265, 1718, 1718, 1718, 1727, 1718, 2026, 1718, 1669, 1718, 1727, 1718, 1073, 1099, 1787, 1253, 1727, 393, 1787, 2191, 1787, 923, 1815, 1815, 1973, 1877, 1610, 1936, 1936, 1942, 1942, 1341, 947, 195, 858, 1718, 2032, 1787, 189, 2081, 2103, 1529, 1787, 2147, 2032, 1529, 1676]
[13, 13, 93, 133, 231, 233, 233, 249, 300, 362, 396, 418, 441, 532, 550, 582, 582, 596, 644, 744, 745, 749, 749, 791, 791, 791, 792, 792, 792, 813, 815, 818, 822, 852, 857, 858, 974, 

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

67
0.46853146853146854
